TP2: Premier réseau de neurone avec PyTorch
=============

Introduction au sujet
-----

L'objectif de ce sujet est de mettre en place un premier réseau de neurone pour classer des fleurs de la base IRIS.

Le code est à écrire en python3 à la suite des questions dans ce fichier. Vous appuierez soit sur le bouton *run cell*, soit sur les touches *Ctrl-Entrée*, à l’intérieur de la zone de saisie, pour lancer l'exécution de vos commandes. Si la commande est en cours d’exécution une étoile apparaît à côté de la zone de saisie de la commande : In [\*]. Une fois le calcul achevé, l'étoile est remplacée par le numéro du run permettant de retrouver par la suite dans quel ordre ont été lancés chaque bloc.

N'hésitez pas à regarder régulièrement la documentation de ces librairies, des exemples d'utilisation accompagnent généralement l'explication de chaque fonction.

Langage utilisé:
- Python 3: https://docs.python.org/3/

Librairie de math:
- Numpy: https://docs.scipy.org/doc/numpy/reference/
- Scipy: https://docs.scipy.org/doc/scipy/reference/

Librairie d'affichage de données:
- Matplotilb: https://matplotlib.org/contents.html

Librairie Pytorch:
- PyTorch: https: https://pytorch.org/docs/stable/

Commencez par importer les librairies nécessaires au TP.

In [1]:
# Import Torch
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

# Import numpy et matplotlib
import numpy as np
import matplotlib.pyplot as plt

# Import de scikit-learn
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection

%matplotlib inline

PyTorch: Premier réseau sur la base IRIS
-----

Nous allons dans ce TP étudier la base *IRIS* et tester un réseau entièrement connecté dessus. 

Commencez par charger la base *IRIS* avec sckit-learn. Vous mettrez les descripteurs des fleurs dans une variable `X` et les labels dans une variable `y`.

In [2]:
iris = sklearn.datasets.load_iris()
X = iris.data
y = iris.target

Vérifiez que le code suivant affiche bien des *True*.

In [3]:
print(X.shape == (150,4))
print(y.shape == (150,))

True
True


Séparez l'ensemble d'apprentissage en deux en utilisant la fonction train_test_split de sckit-learn. Un ensemble d'apprentissage *train* et un ensemble de *test*. Vous prendrez 1/3 des images pour le test.

In [4]:
train,test,y_train,y_test=sk.model_selection.train_test_split(X,y,test_size = 1/3)

Vérifiez les dimensions des données produites:

In [5]:
print(2*len(X)/3,'->',train.shape,y_train.shape)
print(len(X)/3,'->',test.shape,y_test.shape)

100.0 -> (100, 4) (100,)
50.0 -> (50, 4) (50,)


Définissez un classifieur *iris_classifier* correspondant à un réseau entièrement connecté de 3 couches cachées de tailles [10,20,10]. Après chaque couche cachée, vous appliquerez une fonction d'activation de type ReLU. N'oubliez pas la dernière couche de sortie. Vous utiliseriez `torch.nn.Sequential` pour faire cette question.

In [8]:
iris_classifier = torch.nn.Sequential(
    nn.Linear(X.shape[1],10),
    nn.ReLU(),
    nn.Linear(10,20),
    nn.ReLU(),
    nn.Linear(20,10),
    nn.ReLU(),
    nn.Linear(10,3))

Définissez un objet `iter_train` permettant de parcourir la base de donnée d'entrainement avec des batchs aléatoires de taille 32. Vous utiliserez les classes `TensorDataset` et `DataLoader` pour cette question.

In [11]:
train_ds = torch.utils.data.TensorDataset(torch.FloatTensor(train), torch.FloatTensor(y_train))

iter_train = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)

Définissez un objet `iter_test` permettant de parcourir la base de donnée de test avec des batchs de taille 10 concervant l'ordre d'origine des exemples.

In [12]:
test_ds = torch.utils.data.TensorDataset(torch.FloatTensor(test), torch.FloatTensor(y_test))
iter_test = torch.utils.data.DataLoader(test_ds, batch_size=10, shuffle=False)

Définissez une optimiser de type gradient stochastique initialisé avec un taux d'apprentissage de $10^{-2}$.

In [13]:
optimizer = torch.optim.Adam(iris_classifier.parameters(), lr = 0.01)

Définissze un critère de type cross-entropie qui sera utilisé comme fonction de coût optimisant notre réseau.

In [14]:
loss_func = torch.nn.CrossEntropyLoss()

Effectuez 100 époques d'apprentissage du classifieur `iris_classifier` avec les données de `iter_train`. Vous utiliserez pour celà un algorithme de gradient stochastique avec une fonction de coût de type cross-entropie.

In [18]:
for i in range(100):
    iris_classifier.train()
    running_loss = 0.0
    for x, y in iter_train:
        outputs = iris_classifier(x)
        loss = loss_func(outputs, y.type(torch.LongTensor))
        loss.backward() # calcul des gradients
        optimizer.step() # mise a jours des poids
        optimizer.zero_grad() #mise a 0 des gradients
        
    if i%10==0:
        print("loss : ", i,"/",100,loss.item())
print('Finished Training')

loss :  0 / 100 0.1233542263507843
loss :  10 / 100 0.0008796900510787964
loss :  20 / 100 0.024071089923381805
loss :  30 / 100 0.00015171038103289902
loss :  40 / 100 0.0033867498859763145
loss :  50 / 100 0.000732307496946305
loss :  60 / 100 0.012136667035520077
loss :  70 / 100 0.0014039475936442614
loss :  80 / 100 0.007681097835302353
loss :  90 / 100 0.008869434706866741
Finished Training


Evaluez les performances du réseau appris à la  question précédente sur les données de test de `iter_test`. Pour faire  cette  question vous calculerez dans une boucle le nombre de fois que l'algorithme s'est trompé sur la base de test. Pensez à désactiver le calcul des gradients sur la base de test afin de pas perturbé avec des données de tests de nouveaux apprentissages de votre réseau.

In [19]:
iris_classifier.eval()

with torch.no_grad():
    total, correct = 0, 0
    for x, y in iter_test:
        outputs = iris_classifier(x)
        _, predicted = outputs.max(1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print(correct/total)
        
iris_classifier.train()

0.94


Sequential(
  (0): Linear(in_features=4, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=3, bias=True)
)

Relancez les lignes effectuant l'apprentissage et l'évaluation. Comment évolue les performances d'apprentissage ? 

## -> plus on relance l'entrainement plus les resultats obtenus sont bons.

Utilisez tensorboard pour visualiser le graphe correspondant à votre réseau et les différentes courbes correspondant à l'apprentissage de ce dernier.

Vous pouvez pour cela:
- Soit installez le plugin tensorboard pour jupyter: pip3 install --user jupyter-tensorboard

Puis vous suivez les informations d'écrit sur la page: https://github.com/lspvic/jupyter_tensorboard
- Soit lancer dans un terminal: tensorboard --logdir=.

Puis vous vous connectez à http://localhost:6006

PyTorch: Définition d'un réseau couche par couche
-----

Nous allons dans cette partie redéfinir le réseau couche par couche.

Définissez une classe `Net` définissant le réseau précédant sans utiliser de `torch.nn.Sequential`. 

In [20]:
 class Net(nn.Module):
    def __init__(self,shape):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(shape,10)
        self.fc2 = nn.Linear(10,20)
        self.fc3 = nn.Linear(20,10)
        self.fc4 = nn.Linear(10,3)
    def forward(self, x):
        out = nn.functional.relu(self.fc1(x))
        out = nn.functional.relu(self.fc2(out))
        out = nn.functional.relu(self.fc3(out))
        out = self.fc4(out)
        return out
net_classifier = Net(4)

Apprenez ce réseau sur les données d'apprentissage de la base IRIS avec un algorithme de descende de gradient de type AdaGrad dont le taux d'apprentissage est de $10^{-2}$ avec une fonction de coût de type cross-entropie.

In [23]:
opti = torch.optim.Adagrad(net_classifier.parameters(),lr=0.01)
loss_func = torch.nn.CrossEntropyLoss()

net_classifier.train()
for epoch in range (101):
    for xb,yb in iter_train:
        output = net_classifier(xb)
        loss = loss_func(output,yb.type(torch.LongTensor))
        
        loss.backward() # calcul des gradients
        opti.step() # mise a jours des poids
        opti.zero_grad() #mise a 0 des gradients
        
    if (epoch%10==0):
        print(epoch,"/100, loss=",loss.item())

0 /100, loss= 0.05393264815211296
10 /100, loss= 0.027893440797924995
20 /100, loss= 0.10496604442596436
30 /100, loss= 0.017542323097586632
40 /100, loss= 0.006252109073102474
50 /100, loss= 0.09634821861982346
60 /100, loss= 0.012993291951715946
70 /100, loss= 0.003913518041372299
80 /100, loss= 0.14153525233268738
90 /100, loss= 0.012239650823175907
100 /100, loss= 0.011983118951320648


Testez le réseau que vous venez d'apprendre sur la base de test.

In [24]:
net_classifier.eval()
correct=0
with torch.no_grad():
    for data,target in iter_test:
        output = net_classifier(data)
        pred = output.argmax(dim = 1, keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()
    
    print("Accuracy sur test: ",100*correct/len(iter_test.dataset),"%")

Accuracy sur test:  98.0 %


Sauvegardez le modèle que vous venez d'apprendre dans un fichier.

In [25]:
torch.save(net_classifier.state_dict(), "./save_net") 

Chargez le réseau que vous venez de sauvegarder et vérifier que les performances sur la base de test n'ont pas changé.

In [26]:
model = Net(4)
model.load_state_dict(torch.load("./save_net"))
model.eval()
correct=0
with torch.no_grad():
    for data,target in iter_test:
        output = net_classifier(data)
        pred = output.argmax(dim = 1, keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()
    
    print("Accuracy sur test: ",100*correct/len(iter_test.dataset),"%")

Accuracy sur test:  98.0 %


Comparez les graphes des deux méthodes dans tensorboard. Retrouvez-vous les même éléments ? Qu'est ce qui diffère entre les deux versions ? 

Comparaison avec un SVM 
----

En utilisant la librairie sckit-learn et le cours de Machine learning du premier semestre, trouver les performances du meilleur SVM sur ces données et comparer les performances avec celle du réseau de neurone.

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import *